In [62]:
import os
if "models" not in os.listdir("."):
    os.chdir("../..")

In [205]:
use_phi = False
use_9b = True
use_g2 = True
big_g2 = True
mhl = False
layer = 16
if use_phi:
    layer = 18
if not use_phi and not use_g2:
    layer = 12
if use_9b:
    layer = 20

sparsity = "335 l0"

results = {}
zero_results = {}

import json

if use_9b:
    data = json.load(open(f"data/l1_sweep_results_gemma2-9b{'_big' if big_g2 else ''}_0.json"))
    results.update(data)

    zero_results = {
        k.split(":")[0]: {"zero": v["Zero"]["0"][1]} for k, v in results.items()
    }
else:
    for i in range(4):
        # if not mhl:
        #     data = json.load(open(f"data/l1_sweep_results_{'phi' if use_phi else 'gemma2' if use_g2 else 'gemma_higher_lr'}{'_big_higher_lr' if big_g2 else ''}_{i}.json"))
        #     results.update(data)

        #     if use_g2:
        #         data = json.load(open(f"data/gemma_2_hl_extra/l1_sweep_results_{'phi' if use_phi else 'gemma2' if use_g2 else 'gemma_higher_lr'}{'_big' if big_g2 else ''}_higher_lr_{i}.json"))
        #         results.update(data)
        # else:
        #     data = json.load(open(f"data/gemma_2_hl_extra/l1_sweep_results_{'phi' if use_phi else 'gemma2_much_higher_lr' if use_g2 else 'gemma_much_higher_lr'}{'_big_higher_lr' if big_g2 else ''}_{i}.json"))
        #     results.update(data)

        #     data = json.load(open(f"data/gemma_2_hl_extra/l1_sweep_results_{'phi' if use_phi else 'gemma2_much_higher_lr' if use_g2 else 'gemma_much_higher_lr'}{'_big_higher_lr' if big_g2 else ''}_2_{i}.json"))
        #     results.update(data)

        data = json.load(open(f"data/l1_sweep_results_{'phi' if use_phi else 'gemma2' if use_g2 else 'gemma'}{'_big' if big_g2 else ''}_lr_{0.3}_sp_{4}_{i}.json"))
        results.update(data)

        data = json.load(open(f"data/l1_sweep_results_{'phi' if use_phi else 'gemma2' if use_g2 else 'gemma'}{'_big' if big_g2 else ''}_zero_{i}.json"))
        zero_results.update(data)

In [206]:
task_names = list([x.split(":")[0] for x in results.keys()])
task_names = [x for x in task_names if not x.startswith("algo")]

In [207]:
len(results)

23

In [208]:

points = []

for task_name in task_names:
    zero_loss = zero_results[task_name]["zero"]
    task_results = results[f"{task_name}:{layer}"]
    
    tv_loss = task_results["TV"]['0'][1]
    # tv_loss = (zero_loss - tv_loss) / zero_loss
    tv_l0 = task_results["SAE reconstruction"]['0'][0]

    methods = ["ITO", "Task vector cleaning"]

    for method in methods:
        data = task_results[method]
        for i, (s, (l0, loss)) in enumerate(data.items()):
            # loss = (zero_loss - loss) / zero_loss
            if method == "Task vector cleaning":
                i = len(data) - i
                method = "TVC"

            if l0 / tv_l0 > 2:
                continue
            points.append({
                "l0": l0 / tv_l0,
                "loss": max([(tv_loss - loss) / zero_loss, -2]),
                "method": method,
                "size": s
            })

import plotly.express as px

title = "Gemma 1"
file_title = "gemma_1"
if use_phi:
    title = "Phi-3"
    file_title = "phi_3"
if use_g2:
    title = f"Gemma 2 2B 16k ({sparsity})"
    file_title = f"gemma_2_16k_{sparsity.split(' ')[0]}"
if big_g2:
    title = f"Gemma 2 2B 65k ({sparsity})"
    file_title = f"gemma_2_65k_{sparsity.split(' ')[0]}"

if use_9b:
    if big_g2:
        title = "Gemma 2 9B 65k"
        file_title = "gemma_2_9b_65k"
    else:
        title = "Gemma 2 9B 16k"
        file_title = "gemma_2_9b_16k"


if mhl:
    # title = "Gemma 2 16k"
    file_title = f"{file_title}_mhl"




fig = px.scatter(points, x="l0", y="loss", color="method", labels=
                 {"loss": "TV loss delta", 
                  "l0": "TV L0 fraction",
                  "method": "Method",
                  "size": "Hyperparameter"
                  }, title=title, width=1200, height=700)


fig.update_layout(width =700, height=350, 
                font_family="Serif", font_size=12, 
                margin=dict(l=5, r=5, t=40, b=0),
)
import plotly.io as pio
pio.write_image(fig, f"micrlhf-progress/images/l0_loss_delta_{file_title}.pdf", width =700, height=350)

fig

In [209]:

points = []

for task_name in task_names:
    zero_loss = zero_results[task_name]["zero"]
    task_results = results[f"{task_name}:{layer}"]
    
    tv_loss = task_results["TV"]['0'][1]
    tv_loss = tv_loss
    print(
        zero_loss, tv_loss, task_name
    )
    tv_l0 = task_results["SAE reconstruction"]['0'][0]

    methods = ["ITO", "Task vector cleaning"]

    # for method in methods:
    data = task_results["Task vector cleaning"]
    for i, (s, (l0, loss)) in enumerate(data.items()):
        # loss = (zero_loss - loss) / zero_loss
        points.append({
            "l0": l0 / tv_l0,
            "loss_zero": (zero_loss - loss) / zero_loss,
            "loss_tv": (tv_loss - loss) / tv_loss,
            "method": method,
            "coeff": s,
            "task": task_name
        })

import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(rows=2, cols=1)

fig.update_layout(width =1200, height=800)

# fig = px.scatter(points, x="coeff", y="loss", color="task", labels=
#                  {"loss": "Relative loss improvement over TV", 
#                   "l0": "TV L0 fraction",
#                   "method": "Method",
#                   "size": "Hyperparameter"
#                   }, title="Phi-3", width=1200, height=600)

for task_name in task_names:
    fig.add_trace(go.Scatter(x=[x["coeff"] for x in points if x["task"] == task_name], y=[x["loss_zero"] for x in points if x["task"] == task_name], mode='markers', name=task_name), row=1, col=1)

for task_name in task_names:
    fig.add_trace(go.Scatter(x=[x["coeff"] for x in points if x["task"] == task_name], y=[x["l0"] for x in points if x["task"] == task_name], mode='markers', name=task_name), row=2, col=1)

# fig = px.scatter(points, x="coeff", y="loss", color="task", labels=
#                  {"loss": "Relative loss improvement over TV", 
#                   "l0": "TV L0 fraction",
#                   "method": "Method",
#                   "size": "Hyperparameter"
#                   }, title="Phi-3", width=1200, height=600)



fig

4.0625 1.875 antonyms
3.265625 3.15625 country_capital
14.8125 14.0625 en_es
13.375 12.625 en_fr
16.125 14.875 en_it
8.125 5.5 es_en
14.1875 11.875 football_player_position
6.3125 4.4375 fr_en
6.46875 4.0625 it_en
9.125 7.71875 location_continent
4.5 3.390625 location_country
8.625 7.875 location_language
9.75 8.25 location_religion
6.6875 5.78125 person_language
12.8125 9.4375 person_profession
2.421875 1.84375 plural_singular
7.375 7.40625 present_simple_gerund
5.21875 4.625 present_simple_past_perfect
7.40625 6.3125 present_simple_past_simple
4.9375 3.875 singular_plural


In [210]:
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(rows=2, cols=1)

fig.update_layout(width=1200, height=800)

x_labels = list(set([x["coeff"] for x in points]))
x_labels = sorted(x_labels, key=lambda x: float(x))

# Iterate over tasks and create consistent colors and legend
for task_name in task_names:
    # Loss plot
    fig.add_trace(
        go.Scatter(
            x=x_labels,
            y=[x["loss_zero"] for x in points if x["task"] == task_name],
            mode='markers',
            name=task_name,
            legendgroup=task_name,  # Ensure consistent legend across subplots
        ),
        row=1,
        col=1
    )
    
    # L0 plot
    fig.add_trace(
        go.Scatter(
            x=x_labels,
            y=[x["l0"] for x in points if x["task"] == task_name],
            mode='markers',
            name=task_name,
            legendgroup=task_name,  # Same legendgroup as above
            showlegend=False  # Disable legend for the second plot
        ),
        row=2,
        col=1
    )

fig.update_layout(
    title=title,
    xaxis_title="L1 Coefficient",
    yaxis_title="Relative Loss Improvement over TV (Row 1)",
    xaxis2_title="L1 Coefficient",
    yaxis2_title="TV L0 Fraction (Row 2)",
)

fig.update_layout(width =700, height=600, 
                font_family="Serif", font_size=12, 
                margin=dict(l=5, r=5, t=40, b=0),
)

fig


In [211]:
import pandas as pd
import numpy as np

# Convert points to a DataFrame for easier aggregation
df = pd.DataFrame(points)

# Group by coefficient and calculate mean and standard deviation for loss and l0
aggregated = df.groupby("coeff").agg(
    mean_loss_zero=("loss_zero", "mean"),
    std_loss_zero=("loss_zero", "std"),
    mean_loss_tv=("loss_tv", "mean"),
    std_loss_tv=("loss_tv", "std"),
    mean_l0=("l0", "mean"),
    std_l0=("l0", "std")
).reset_index()

aggregated = aggregated.sort_values("coeff", key=lambda x: x.astype(float))

# Create a new figure
fig = make_subplots(rows=2, cols=1)

# Add traces for mean loss with error bars
fig.add_trace(
    go.Scatter(
        x=aggregated["coeff"],
        y=aggregated["mean_loss_zero"],
        error_y=dict(
            type="data",
            array=aggregated["std_loss_zero"],  # Standard deviation as error bars
            visible=True
        ),
        mode='markers+lines',
        # name="Mean",
    ),
    row=1,
    col=1
)

# fig.add_trace(
#     go.Scatter(
#         x=aggregated["coeff"],
#         y=aggregated["mean_loss_tv"],
#         error_y=dict(
#             type="data",
#             array=aggregated["std_loss_tv"],  # Standard deviation as error bars
#             visible=True
#         ),
#         mode='markers+lines',
#         # name="Mean Loss Tv",
#     ),
#     row=1,
#     col=1
# )

# Add traces for mean l0 with error bars
fig.add_trace(
    go.Scatter(
        x=aggregated["coeff"],
        y=aggregated["mean_l0"],
        error_y=dict(
            type="data",
            array=aggregated["std_l0"],  # Standard deviation as error bars
            visible=True
        ),
        mode='markers+lines',
        # name="Mean L0 Fraction",
    ),
    row=2,
    col=1
)

# Update layout
fig.update_layout(
    width=1200,
    height=1200,
    title=title,
    xaxis_title="L1 Coefficient",
    yaxis_title="Mean relative loss decrease",
    xaxis2_title="L1 Coefficient",
    yaxis2_title="Mean TV L0 Fraction",
    showlegend=False
)

fig.update_layout(width =300, height=400, 
                font_family="Serif", font_size=12, 
                margin=dict(l=5, r=5, t=40, b=0),
)



pio.write_image(fig, f"micrlhf-progress/images/l1_sweep_{file_title}.pdf", width =300, height=400)
fig


In [176]:
import pandas as pd                                 